# EDA & Model Development 

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import random

## Initial Setup

In [31]:
#https://raw.githubusercontent.com/jchristo12/fantasy_football/master/data/full_data.csv
df = pd.read_csv('https://github.com/jchristo12/fantasy_football/blob/master/data/full_data.csv?raw=true')

In [38]:
df.columns

Index(['pk', 'gid', 'seas', 'wk', 'player', 'fname', 'lname', 'full_name',
       'team', 'pos1',
       ...
       'humd', 'ou', 'sprv', 'ptsv', 'ptsh', 'udog', 'gen_cond', 'udog_binary',
       'def_team', 'f_pts'],
      dtype='object', length=161)

In [32]:
#remove rows that have NaN for the shifted variables
df_clean = df[~df.loc[:,'seas_pa':'seas_tdret'].isna().all(axis=1)]

##### We are only focused on offensive players right now. Therefore, we will specifiy the positions we want to retain in the data and remove everyone else

In [53]:
#store positions we are concerned about; will use these to filter out 
pos_of_interest = ['QB', 'RB', 'WR', 'TE']
#filter out positions we don't care about
df_clean2 = df_clean[df_clean['pos1'].isin(pos_of_interest)]

##### Convert each column to the appropriate data type (i.e. make the categorical data categories)

In [49]:
#set the column types
col_dtypes = {'category': ['seas', 'wk', 'pos1', 'team', 'udog', 'dv', 'v', 'h', 'day', 'cond', 'stad', 'wdir',
                          'surf', 'gen_cond', 'def_team']}
#flip the key and values around so they will work in the argument for 'astype()'
col_dtypes_alt = {old: new for new, old_all in col_dtypes.items() for old in old_all}

In [52]:
df_clean2 = df_clean2.astype(col_dtypes_alt)

##### Segment out the WR data

In [54]:
df_wr = df_clean2.loc[df_clean2['pos1']=='WR', :]

## EDA

Only focusing on the WR data right now. Will need to build this so that each action is extendable to the other positions

### Training/Test split

In [71]:
from sklearn.model_selection import train_test_split

In [72]:
#set the random seed for reproducability
random.seed(837)

In [73]:
#break out the data between training and test
train_wr, test_wr = train_test_split(df_wr, train_size=0.70, test_size=0.30, shuffle=True)

In [ ]:
train_wr

### Missing Data analysis

In [83]:
missing_data_pct = train_wr.isna().sum() / train_wr.shape[0]
missing_data_pct[missing_data_pct > 0.25].sort_values(ascending=False)

roll_td_to_int           0.994871
seas_td_to_int           0.990896
roll_yds_per_comp        0.989700
seas_yds_per_comp        0.981921
roll_comp_pct            0.977989
seas_comp_pct            0.962602
career_td_to_int         0.849895
career_yds_per_comp      0.831474
career_comp_pct          0.784118
roll_ryds_per_carry      0.742745
roll_carry_to_td         0.742189
roll_ret_to_td           0.681027
roll_avg_ret             0.680814
roll_carry_to_fuml       0.677523
seas_ryds_per_carry      0.660213
seas_carry_to_td         0.659401
seas_ret_to_td           0.650083
seas_avg_ret             0.649699
career_ret_to_td         0.597171
career_avg_ret           0.597085
seas_carry_to_fuml       0.571825
career_ryds_per_carry    0.322819
career_carry_to_td       0.322392
humd                     0.271573
wdir                     0.261358
dtype: float64